# Building a protein-membrane molecular system (GPCR)

Specifically we are building the $\mu$ opiod receptor using the pdbid 4dkl from the OPM database.

In [1]:
%load_ext autoreload 
%autoreload 2
from htmd.console import *
path = home() + '/data/building-protein-membrane/'

HTMD Announcement
16-Oct-2015: New version (0.0.9) of HTMD available please update with 
	     conda update htmd



/shared/sdoerr/Software/anaconda3/lib/python3.4/site-packages/pyEMMA-2.0.1-py3.4-linux-x86_64.egg/pyemma/coordinates/util/stat.py:31: DeprecationWarning: Call to deprecated function hist. Called from pyemma.coordinates.util.stat line 31. Please use pyemma.coordinates.histogram()
  def hist(transform, dimensions, nbins):


In [2]:
charmm.listFiles()

---- Topologies files list: /shared/sdoerr/Work/pyHTMD/htmd/builder/charmmfiles/top/ ----
top/top_all22star_prot.rtf
top/top_all36_carb.rtf
top/top_all36_lipid.rtf
top/top_all36_prot.rtf
top/top_water_ions.rtf
top/top_all36_cgenff.rtf
top/top_all36_na.rtf
---- Parameters files list: /shared/sdoerr/Work/pyHTMD/htmd/builder/charmmfiles/par/ ----
par/par_all22star_prot.prm
par/par_all36_carb.prm
par/par_all36_lipid.prm
par/par_all36_prot.prm
par/par_all36_cgenff.prm
par/par_all36_na.prm
par/par_water_ions.prm


In [3]:
prj = 'mornap'
salt = 0.15
topos  = ['top/top_all36_prot.rtf','top/top_all36_lipid.rtf', 'top/top_water_ions.rtf']
params = ['par/par_all36_prot.prm','par/par_all36_lipid.prm', 'par/par_water_ions.prm']

In [4]:
prot = Molecule(path + '4dkl.pdb')#from opm
prot.filter('protein and noh and chain B or water within 5 of (chain B and protein)')
prot = segmentgaps(prot,'protein','P') 
pcenter = mean(prot.get('coords','protein'),axis=0)
unique(prot.get('segid'))

Found  segment between resid  65  and  263
Found  segment between resid  270  and  352


array(['A2', 'B2', 'P1', 'P2'], dtype=object)

In [5]:
memb = Molecule(path + 'membrane.pdb')
mcenter = mean(memb.get('coords'),axis=0)
memb.moveBy(pcenter-mcenter)
mol = embed(prot,memb)

In [6]:
coo = mol.get('coords','lipids or protein')
m = amin(coo,axis=0) + [0,0,-15]
M = amax(coo,axis=0) + [0,0,15]
mol = solvate(mol, minmax=vstack((m,M)))

In [7]:
topos  = ['top/top_all36_prot.rtf','top/top_all36_lipid.rtf', 'top/top_water_ions.rtf']
params = ['par/par_all36_prot.prm','par/par_all36_lipid.prm', 'par/par_water_ions.prm']
mol = charmm.build(mol, topo=topos, param=params, outdir='/tmp/buil',saltconc=0.15)

Bond between A: [serial 17227 resid 140 resname CYS chain B segid P1]
             B: [serial 17837 resid 217 resname CYS chain B segid P1]



In [8]:
mol.view()